In [1]:
# import rasterio
import numpy as np
import matplotlib.pyplot as plt
# from shapely.geometry import Polygon, box
# import geopandas as gpd
import ee
import geemap
# from rasterio import features
import json
import os
import folium
# ee.Authenticate()
ee.Initialize()


In [2]:
from tools import filteringModule

In [3]:
help(ee.Filter)

Help on class Filter in module ee.filter:

class Filter(ee.computedobject.ComputedObject)
 |  Filter(*args, **kwargs)
 |  
 |  An object to represent collection filters.
 |  
 |  Method resolution order:
 |      Filter
 |      ee.computedobject.ComputedObject
 |      ee.encodable.Encodable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  Not(self)
 |      Returns the opposite of this filter.
 |      
 |      Returns:
 |        The negated filter, which will match iff this filter doesn't.
 |  
 |  __init__(self, opt_filter=None)
 |      Construct a filter.
 |      
 |      This constructor accepts the following args:
 |        1) Another filter.
 |        2) An array of filters (which are implicitly ANDed together).
 |        3) A ComputedObject returning a filter. Users shouldn't be making these;
 |           they're produced by the generator functions below.
 |      
 |      Args:
 |        opt_filter: Optional filter to add.
 |  
 |  predicateCount(self)
 |      Return

In [4]:
# folder = (new Function('return __dirname'))()
# filteringModule = require('filters.js')
city_name = 'Bangalore Urban'
admin2 = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level2')
selected = admin2.filter(ee.Filter.eq('ADM2_NAME', city_name))
geometry = selected.geometry()
geom = geometry.bounds()
# print('Bangalore Bounds:', geom)

year_start = '2020'
year_end = '2023'
image_before_start = '2015-11-01'
image_before_end = '2016-03-01'
image_after_start = '2023-01-01'
image_after_end = '2023-04-01'
DW_bands = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops',
    'shrub_and_scrub', 'built', 'bare', 'snow_and_ice']
image_col_ = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1"). \
            filterDate(image_before_start, image_after_end) \
            .filterBounds(geom) \
            
monthlyMedians = filteringModule.computeMonthlyMedians(image_col_)
monthlyMedians = filteringModule.forward_fill(monthlyMedians)
# print(monthlyMedians)
# print(ee.Image(monthlyMedians.toList(monthlyMedians.size()).get(30)).bandNames().size())
monthlyMedians = monthlyMedians.map(filteringModule.addBandnumber).filter(ee.Filter.gt('numBands',0))
band = 'label'
dw =  monthlyMedians.select(band)
beforeDw = dw.filterDate(image_before_start, image_before_end).mean()
afterDw = dw.filterDate(image_after_start, image_after_end).mean()
newUrban = beforeDw.lt(0.3).And(afterDw.gt(0.7))
lon, lat = geometry.centroid().getInfo()['coordinates']
# Map = folium.Map(location= [lat, lon], zoom_start=12)

Map = geemap.Map()
changeVisParams = {'min': 0, 'max': 1, 'palette': ['white', 'red']}
Map.addLayer(dw.first().mask().clip(geometry), changeVisParams, 'Urban')
# Map.addLayer(dw.first().mask().Not(), {}, 'Urban')
Map.addLayer(newUrban, changeVisParams, "New Urban")
# Map.addLayer(monthlyMedians.first().select(['built', 'trees', 'water']).clip(geom), {}, 'Bangalore')
Map.centerObject(geometry, 12)
Map

Map(center=[12.942208526432214, 77.5868323785926], controls=(WidgetControl(options=['position', 'transparent_b…

In [7]:
# Function to calculate the statistics for an image
def calculateStatistics(image):
    statistics = image.gt(0.7).reduceRegion(
        reducer=ee.Reducer.sum(),  # Use the desired reducer function, such as mean, sum, max, etc.
        geometry=geometry,
        scale=20  # Set the appropriate scale based on the image resolution
    )
    return image.set(statistics)

# Apply the calculateStatistics function over the image collection
image_collection_with_stats = dw.map(calculateStatistics)

# Get the resulting image collection with statistics
image_collection_with_stats

In [36]:
img1.select('label').gt(0.7).reduceRegion(
        reducer=ee.Reducer.sum(),  # Use the desired reducer function, such as mean, sum, max, etc.
        geometry=geometry,
        scale=20  # Set the appropriate scale based on the image resolution
    ).getInfo()

{'label': 4158549.749019596}

In [16]:
image_col.first()

In [39]:
img2

In [38]:
img1 = ee.Image(image_col_.toList(image_col_.size()).get(16))
img2 = ee.Image(image_col_.toList(image_col_.size()).get(17))
Map = geemap.Map()
changeVisParams = {'min': 0, 'max': 1, 'palette': ['white', 'red']}
# Map.addLayer(img1.clip(geometry))
# Map.addLayer(dw.first().mask().Not(), {}, 'Urban')
null_mask = img2.mask().Not()
# Map.addLayer(null_mask)
Map.addLayer(img2.unmask(1).where(null_mask, img1).clip(geometry))
# Map.addLayer(img1)
# Map.addLayer(monthlyMedians.first().select(['built', 'trees', 'water']).clip(geom), {}, 'Bangalore')
Map.centerObject(geometry, 12)
Map

Map(center=[12.942208526432214, 77.5868323785926], controls=(WidgetControl(options=['position', 'transparent_b…

In [30]:
reduced = dw.map(lambda image: ee.Feature(None, image.eq(6).reduceRegion(reducer = ee.Reducer.sum(), geometry= geom, scale = 20)\
                                          .set('month', image.get('month'))\
                                           .set('year', image.get('year')) ))
area_collection_dict = reduced.getInfo()

In [31]:
feature_list = area_collection_dict.get('features')
feature_count = len(feature_list)
areas = (np.array([i.get('properties').get(band) for i in feature_list])*20)**2
month_data = np.array([i.get('properties').get('month') for i in feature_list])
year_data = np.array([i.get('properties').get('year') for i in feature_list])
areas

array([2.28707292e+15, 3.24806151e+15, 3.55603820e+15, 3.67621614e+15,
       4.11814356e+15, 7.55958646e+07, 0.00000000e+00, 3.98269529e+15,
       3.55216151e+15, 3.81217347e+15, 3.68794765e+15, 3.58044736e+15,
       5.37121094e+14, 3.73384640e+15, 9.70380618e+14, 1.54049761e+13,
       2.53699863e+09, 6.51734847e+08, 2.69450114e+15, 3.17463921e+15,
       3.95633693e+15, 4.07643453e+15, 4.26030552e+15, 5.24168163e+15,
       2.35195924e+15, 1.30830571e+12, 3.12722184e+05, 5.49664568e+12,
       4.02151239e+15, 4.61797107e+15, 4.22757063e+15, 4.34700026e+15,
       4.75975706e+15, 4.67687509e+15, 4.96572642e+15, 5.99737840e+15,
       6.80385040e+12, 3.85860208e+03, 0.00000000e+00, 8.25217207e+11,
       1.18749701e+09, 2.95994437e+15, 6.30649557e+13, 4.92568557e+15,
       4.85629197e+15, 5.30408837e+15, 5.60710133e+15, 6.06151717e+13,
       1.26881596e+14, 4.24009706e+15, 4.76298308e+13, 4.63189845e+15,
       5.13501575e+15, 5.06050774e+15, 5.36447463e+15, 1.67628727e+15,
      

In [17]:
# dir_path = f"data/{city_name}/{band}"
# os.makedirs(dir_path, exist_ok=True)
# with open(f"{dir_path}/areas.json", 'w') as dest:
#     json.dump(feature_list, dest, indent = 6)

In [18]:
unique_year = np.sort(np.unique(year_data))
yearly_data = [np.max(areas[year_data == i][0:5]) for i in unique_year]
yearly_data

[2287072921498875.5,
 4118143558869179.5,
 3733846396797620.0,
 5241681626583168.0,
 5997378403883456.0,
 5607101331612256.0,
 5364474628911943.0,
 5513405313211407.0,
 5875898830968877.0]

In [19]:
import pandas as pd
import seaborn as sns

In [20]:
df = pd.DataFrame({"year": year_data, "month": month_data, "areas": areas})
df

,year,month,areas
0,2015,12,2.287073e+15
1,2016,1,3.248062e+15
2,2016,2,3.556038e+15
3,2016,4,3.676216e+15
4,2016,5,4.118144e+15
...,...,...,...
71,2022,11,9.324531e+14
72,2022,12,5.298825e+15
73,2023,1,5.536914e+15
74,2023,2,5.848084e+15


In [21]:
import plotly.express as px

In [22]:
px.line(x=unique_year, y=yearly_data)

In [23]:
df_filtered = df[df.month.isin([1,2,3, 4])]
px.line(df, x='year', y='areas', color = 'month')